# Prompt templates and Prompt Values

In [2]:
from langchain_core.prompts import PromptTemplate, AIMessagePromptTemplate

In [1]:
TEMPLATE = """
System:
{description}

Human:
I've recently adopted a {pet}.
Could you suggest some {pet} names?
"""

In [3]:
prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [4]:
prompt_template

PromptTemplate(input_variables=['description', 'pet'], input_types={}, partial_variables={}, template="\nSystem:\n{description}\n\nHuman:\nI've recently adopted a {pet}.\nCould you suggest some {pet} names?\n")

In [5]:
prompt_value = prompt_template.invoke({'description': """The chatbot should reluctantly answer questions with sarcastic responses.""", "pet":'dog'})

In [6]:
prompt_value

StringPromptValue(text="\nSystem:\nThe chatbot should reluctantly answer questions with sarcastic responses.\n\nHuman:\nI've recently adopted a dog.\nCould you suggest some dog names?\n")

## Chat Prompt templates and Chat Prompt values

In [7]:
%load_ext dotenv
%dotenv

In [24]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import AIMessagePromptTemplate, HumanMessagePromptTemplate,ChatPromptTemplate, FewShotChatMessagePromptTemplate, SystemMessagePromptTemplate

In [18]:
chat = ChatOpenAI(model_name="gpt-4o-mini", seed=365, temperature=0, max_tokens=100)

In [10]:
SYSTEM_TEMPLATE = "{description}"
HUMAN_TEMPLATE = """I've recently adopted a {pet}.
Could you suggest some {pet} names?"""

In [11]:
message_template_system = SystemMessagePromptTemplate.from_template(template=SYSTEM_TEMPLATE)
message_template_system

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['description'], input_types={}, partial_variables={}, template='{description}'), additional_kwargs={})

In [12]:
message_template_human =HumanMessagePromptTemplate.from_template(template=HUMAN_TEMPLATE)
message_template_human

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template="I've recently adopted a {pet}.\nCould you suggest some {pet} names?"), additional_kwargs={})

In [14]:
chat_template = ChatPromptTemplate.from_messages([message_template_system, message_template_human])
chat_template

ChatPromptTemplate(input_variables=['description', 'pet'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['description'], input_types={}, partial_variables={}, template='{description}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template="I've recently adopted a {pet}.\nCould you suggest some {pet} names?"), additional_kwargs={})])

In [15]:
chat_value = chat_template.invoke({'description': """The chatbot should reluctantly answer questions with sarcastic responses.""", "pet":'dog'})

In [16]:
chat_value

ChatPromptValue(messages=[SystemMessage(content='The chatbot should reluctantly answer questions with sarcastic responses.', additional_kwargs={}, response_metadata={}), HumanMessage(content="I've recently adopted a dog.\nCould you suggest some dog names?", additional_kwargs={}, response_metadata={})])

In [19]:
response = chat.invoke(chat_value)

In [21]:
response.content

'Oh, sure, because naming a dog is the most challenging task in the world. How about "Bark Twain" or "Sir Waggington"? Or you could go with the classic "Doggo." I mean, the possibilities are endless, right?'

# Few-Shot Chat Message Prompt Templates 

In [37]:
AI_TEMPLATE = "{response}"
HUMAN_TEMPLATE = """I've recently adopted a {pet}.
Could you suggest some {pet} names?"""

In [38]:
message_template_ai = AIMessagePromptTemplate.from_template(template=AI_TEMPLATE)
message_template_human = HumanMessagePromptTemplate.from_template(template=HUMAN_TEMPLATE)

In [39]:
example_template = ChatPromptTemplate.from_messages([message_template_ai, message_template_human])

In [40]:
examples = [{'pet':'dog', 
             'response':'''Oh, absolutely. Because nothing screams "I'm a responsible pet owner" 
like asking a chatbot to name your new furball. How about "Bark Twain" (if it's a literary hound)? '''}, 
            
            {'pet':'cat', 
             'response':'''Oh, absolutely. Because nothing screams "I'm a unique and creative individual" 
             like asking a chatbot to name your cat. How about "Furry McFurFace", "Sir Meowsalot", or "Catastrophe"? '''}, 
            
            {'pet':'fish', 
             'response':
             '''Oh, absolutely. Because nothing screams "I'm a fun and quirky pet owner" 
             like asking a chatbot to name your fish. How about "Fin Diesel", "Gill Gates", or "Bubbles"?'''}]

In [41]:
few_shot_prompt = FewShotChatMessagePromptTemplate(examples=examples, example_prompt=example_template, input_variables = ['pet'])

In [46]:
chat_template = ChatPromptTemplate.from_messages([few_shot_prompt, message_template_human])

In [47]:
chat_value = chat_template.invoke({'pet':'rabbit'})
chat_value

ChatPromptValue(messages=[AIMessage(content='Oh, absolutely. Because nothing screams "I\'m a responsible pet owner" \nlike asking a chatbot to name your new furball. How about "Bark Twain" (if it\'s a literary hound)? ', additional_kwargs={}, response_metadata={}), HumanMessage(content="I've recently adopted a dog.\nCould you suggest some dog names?", additional_kwargs={}, response_metadata={}), AIMessage(content='Oh, absolutely. Because nothing screams "I\'m a unique and creative individual" \n             like asking a chatbot to name your cat. How about "Furry McFurFace", "Sir Meowsalot", or "Catastrophe"? ', additional_kwargs={}, response_metadata={}), HumanMessage(content="I've recently adopted a cat.\nCould you suggest some cat names?", additional_kwargs={}, response_metadata={}), AIMessage(content='Oh, absolutely. Because nothing screams "I\'m a fun and quirky pet owner" \n             like asking a chatbot to name your fish. How about "Fin Diesel", "Gill Gates", or "Bubbles"?',

In [48]:
for i in chat_value.messages:
    print(f'{i.typfe}: {i.content}\n')

ai: Oh, absolutely. Because nothing screams "I'm a responsible pet owner" 
like asking a chatbot to name your new furball. How about "Bark Twain" (if it's a literary hound)? 

human: I've recently adopted a dog.
Could you suggest some dog names?

ai: Oh, absolutely. Because nothing screams "I'm a unique and creative individual" 
             like asking a chatbot to name your cat. How about "Furry McFurFace", "Sir Meowsalot", or "Catastrophe"? 

human: I've recently adopted a cat.
Could you suggest some cat names?

ai: Oh, absolutely. Because nothing screams "I'm a fun and quirky pet owner" 
             like asking a chatbot to name your fish. How about "Fin Diesel", "Gill Gates", or "Bubbles"?

human: I've recently adopted a fish.
Could you suggest some fish names?

human: I've recently adopted a rabbit.
Could you suggest some rabbit names?



In [49]:
response = chat.invoke(chat_value)

In [53]:
print(response.content)

Congratulations on your new rabbit! Here are some fun name suggestions:

1. **Thumper**
2. **BunBun**
3. **Clover**
4. **Nibbles**
5. **Hopper**
6. **Flopsy**
7. **Peanut**
8. **Snowball**
9. **Cinnabun**
10. **Luna**

Feel free to mix and match or modify these names to find the perfect fit for your new furry friend!


## LLM Chain

In [54]:
from langchain.chains.llm import LLMChain

In [57]:
chain = chat_template | chat
response = chain.invoke({'pet':'fish'})

In [58]:
response

AIMessage(content='Of course! Here are some fun and creative names for your new fish:\n\n1. **Finley**\n2. **Bubbles**\n3. **Nemo**\n4. **Gilly**\n5. **Swim Shady**\n6. **Captain Fluff**\n7. **Marina**\n8. **Goldie**\n9. **Splash**\n10. **Dory**\n\nFeel free to mix and match or modify them to find the perfect name for your aquatic friend!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 217, 'total_tokens': 313, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BeywrYacd9D5pQNGvHcDjnGjJ2cHi', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1b0e4e56-ee4f-4ab5-8bd8-95f7b26275c4-0', usage_metadata={'input_tokens': 217, 'output_tokens': 96, 'tota